In [1]:
import duckdb
conn = duckdb.connect("bvg_data.db")

In [2]:
conn.execute("SELECT * FROM m13_arrivals LIMIT 3").fetch_df()

,trip_id,line_name,product,station_id,station_name,direction,scheduled_time,actual_time,delay,delay_calc,occupancy,remarks
0,1|64465|30|86|23012025,M13,tram,900140011,Antonplatz (Berlin),NaN,2025-01-23T11:22:00+01:00,2025-01-23T11:22:00+01:00,0.0,0.0,,"[""Bicycle conveyance""]"
1,1|64500|30|86|23012025,M13,tram,900140011,Antonplatz (Berlin),NaN,2025-01-23T11:26:00+01:00,2025-01-23T11:25:00+01:00,-60.0,-60.0,,"[""Bicycle conveyance""]"
2,1|64465|31|86|23012025,M13,tram,900140011,Antonplatz (Berlin),NaN,2025-01-23T11:32:00+01:00,2025-01-23T11:33:00+01:00,60.0,60.0,,"[""Bicycle conveyance""]"


In [3]:
conn.execute("SELECT * FROM m13_departures LIMIT 3").fetch_df()

,trip_id,line_name,product,station_id,station_name,direction,scheduled_time,actual_time,delay,delay_calc,occupancy,remarks
0,1|64465|30|86|23012025,M13,tram,900140011,Antonplatz (Berlin),S Warschauer Straße,2025-01-23T11:22:00+01:00,2025-01-23T11:22:00+01:00,0.0,0.0,low,"[""Bicycle conveyance""]"
1,1|64500|30|86|23012025,M13,tram,900140011,Antonplatz (Berlin),"Wedding, Virchow-Klinikum",2025-01-23T11:26:00+01:00,2025-01-23T11:25:00+01:00,-60.0,-60.0,low,"[""Bicycle conveyance""]"
2,1|64465|31|86|23012025,M13,tram,900140011,Antonplatz (Berlin),S Warschauer Straße,2025-01-23T11:32:00+01:00,2025-01-23T11:33:00+01:00,60.0,60.0,low,"[""Bicycle conveyance""]"


## How punctual is the M13 tram at Schönhauser Allee/Bornholmer Straße and Antonplatz?

In [23]:
# Count total records for M13 at each station
conn.execute("""
                SELECT
                    station_name,
                    COUNT(DISTINCT trip_id) as trips,
                    COUNT(*) as total_records,
                    COUNT(*) - COUNT(DISTINCT trip_id) as duplicate_trips
                FROM m13_arrivals
                GROUP BY station_name
                ORDER BY station_name;
                """).fetch_df()

,station_name,trips,total_records,duplicate_trips
0,Antonplatz (Berlin),3442,3530,88
1,Schönhauser Allee/Bornholmer Str. (Berlin),3471,3592,121


In [24]:
# Find the date range of your data
conn.execute("""
                SELECT
                    CAST(scheduled_time AS DATE) as date,
                    COUNT(DISTINCT trip_id) as records_per_day
                FROM m13_arrivals
                GROUP BY CAST(scheduled_time AS DATE)
                ORDER BY date;
                """).fetch_df()

,date,records_per_day
0,2025-01-23,60
1,2025-01-24,230
2,2025-01-25,224
3,2025-01-26,212
4,2025-01-27,228
5,2025-01-28,236
6,2025-01-29,220
7,2025-01-30,235
8,2025-01-31,228
9,2025-02-01,233


### Cleaning
I will need to filter out the following dates
- 2025-01-23 and 2025-02-08	due to incomplete data collection
- 2025-01-27 because there was a BVG strike hence no trams were running

In [27]:
conn.execute("""
                SELECT
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    -- Extract the difference in seconds for analysis
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds,
                    CASE 
                        WHEN ABS(EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME)))) <= 60 
                            THEN 'punctual'
                        WHEN EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) < -60 
                            THEN 'early'
                        ELSE 'late'
                    END as punctuality_status,
                    ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time ORDER BY actual_time DESC) as row_num
                FROM m13_arrivals
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
                --  AND trip_id = '1|61373|1|86|2022025'
                ORDER BY row_num DESC
                LIMIT 10; 
                """).fetch_df()

,trip_id,station_name,scheduled_time,actual_time,delay_seconds,punctuality_status,row_num
0,1|60858|2|86|25012025,Antonplatz (Berlin),2025-01-25 06:58:00,2025-01-25 06:58:00,0.0,punctual,2
1,1|60861|6|86|25012025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-01-25 22:29:00,2025-01-25 22:29:00,0.0,punctual,2
2,1|60889|9|86|28012025,Antonplatz (Berlin),2025-01-28 06:56:00,2025-01-28 06:56:00,0.0,punctual,2
3,1|60898|2|86|26012025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-01-26 20:58:00,2025-01-26 20:58:00,0.0,punctual,2
4,1|60899|0|86|25012025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-01-25 22:28:00,2025-01-25 22:29:00,60.0,punctual,2
5,1|61320|8|86|4022025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-02-04 06:56:00,2025-02-04 06:57:00,60.0,punctual,2
6,1|61402|19|86|31012025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-01-31 15:56:00,2025-01-31 15:56:00,0.0,punctual,2
7,1|61408|2|86|2022025,Schönhauser Allee/Bornholmer Str. (Berlin),2025-02-02 20:58:00,2025-02-02 20:59:00,60.0,punctual,2
8,1|63941|12|86|29012025,Antonplatz (Berlin),2025-01-29 07:26:00,2025-01-29 07:26:00,0.0,punctual,2
9,1|58966|9|86|7022025,Antonplatz (Berlin),2025-02-07 21:58:00,2025-02-07 21:58:00,0.0,punctual,2


In [28]:
# Calculate the overall punctuality distribution (percentage of early/punctual/late) for each station
conn.execute("""
                WITH per_trip_punctuality AS (
                SELECT
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    -- Extract the difference in seconds for analysis
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds,
                    CASE 
                        WHEN ABS(EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME)))) <= 60 
                            THEN 'punctual'
                        WHEN EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) < -60 
                            THEN 'early'
                        ELSE 'late'
                    END as punctuality_status,
                    ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time 
                                        ORDER BY CAST(actual_time AS DATETIME) DESC) as row_num
                FROM m13_arrivals
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
                -- ORDER BY delay_seconds DESC
                )

                SELECT
                    station_name,
                    COUNT(*) as trips,
                    SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END) as late_trips,
                    SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END) as early_trips,
                    SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END) as punctual_trips,
                    (100 * SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END)) / COUNT(*) as percent_punctual
                FROM per_trip_punctuality
                WHERE row_num = 1 -- filter out duplicate records
                GROUP BY station_name
                """).fetch_df()

,station_name,trips,late_trips,early_trips,punctual_trips,percent_punctual
0,Schönhauser Allee/Bornholmer Str. (Berlin),3104,509.0,1199.0,1396.0,44.974227
1,Antonplatz (Berlin),3080,285.0,868.0,1927.0,62.564935


In [8]:
# Create hourly analysis to see if punctuality varies throughout the day
conn.execute("""
                WITH per_trip_punctuality AS (
                SELECT
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    -- Extract the difference in seconds for analysis
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds,
                    CASE 
                        WHEN ABS(EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME)))) <= 60 
                            THEN 'punctual'
                        WHEN EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) < -60 
                            THEN 'early'
                        ELSE 'late'
                    END as punctuality_status,
                    ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time 
                                        ORDER BY CAST(actual_time AS DATETIME) DESC) as row_num
                FROM m13_arrivals
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
                -- ORDER BY delay_seconds DESC
                )

                SELECT
                    station_name,
                    EXTRACT(HOUR FROM scheduled_time) as hour_of_day,
                    COUNT(*) as trips,
                    SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END) as late_trips,
                    SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END) as early_trips,
                    SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END) as punctual_trips,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_punctual,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_late,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_early,
                    ROUND(AVG(delay_seconds), 2) as avg_delay_in_arrival_seconds
                FROM per_trip_punctuality
                WHERE row_num = 1
                GROUP BY station_name, EXTRACT(HOUR FROM scheduled_time)
                ORDER BY station_name, EXTRACT(HOUR FROM scheduled_time)
                LIMIT 10;
                """).fetch_df()

,station_name,hour_of_day,trips,late_trips,early_trips,punctual_trips,percent_punctual,percent_late,percent_early,avg_delay_in_arrival_seconds
0,Antonplatz (Berlin),0,61,6.0,22.0,33.0,54.10,9.84,36.07,-116.79
1,Antonplatz (Berlin),1,48,5.0,12.0,31.0,64.58,10.42,25.00,-107.73
2,Antonplatz (Berlin),2,52,2.0,19.0,31.0,59.62,3.85,36.54,-144.71
3,Antonplatz (Berlin),3,75,1.0,20.0,54.0,72.00,1.33,26.67,-112.00
4,Antonplatz (Berlin),4,100,3.0,26.0,71.0,71.00,3.00,26.00,-75.00
5,Antonplatz (Berlin),5,123,4.0,38.0,81.0,65.85,3.25,30.89,-86.34
6,Antonplatz (Berlin),6,123,14.0,32.0,77.0,62.60,11.38,26.02,-76.10
7,Antonplatz (Berlin),7,149,27.0,36.0,86.0,57.72,18.12,24.16,-32.62
8,Antonplatz (Berlin),8,142,20.0,38.0,84.0,59.15,14.08,26.76,-36.34
9,Antonplatz (Berlin),9,148,10.0,35.0,103.0,69.59,6.76,23.65,-49.46


In [9]:
# Applying the same query for Departures
# Create hourly analysis to see if punctuality varies throughout the day
conn.execute("""
                WITH per_trip_punctuality AS (
                SELECT
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    -- Extract the difference in seconds for analysis
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds,
                    CASE 
                        WHEN ABS(EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME)))) <= 60 
                            THEN 'punctual'
                        WHEN EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) < -60 
                            THEN 'early'
                        ELSE 'late'
                    END as punctuality_status,
                    ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time 
                                        ORDER BY CAST(actual_time AS DATETIME) DESC) as row_num
                FROM m13_departures
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
                -- ORDER BY delay_seconds DESC
                )

                SELECT
                    station_name,
                    EXTRACT(HOUR FROM scheduled_time) as hour_of_day,
                    COUNT(*) as trips,
                    SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END) as late_trips,
                    SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END) as early_trips,
                    SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END) as punctual_trips,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_punctual,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_late,
                    ROUND((100.0 * SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END)) / COUNT(*), 2) as percent_early,
                    ROUND(AVG(delay_seconds), 2) as avg_delay_in_arrival_seconds
                FROM per_trip_punctuality
                WHERE row_num = 1
                GROUP BY station_name, EXTRACT(HOUR FROM scheduled_time)
                ORDER BY station_name, EXTRACT(HOUR FROM scheduled_time)
                LIMIT 10;
                """).fetch_df()

,station_name,hour_of_day,trips,late_trips,early_trips,punctual_trips,percent_punctual,percent_late,percent_early,avg_delay_in_arrival_seconds
0,Antonplatz (Berlin),0,50,5.0,16.0,29.0,58.00,10.00,32.00,-122.61
1,Antonplatz (Berlin),1,48,5.0,12.0,31.0,64.58,10.42,25.00,-107.73
2,Antonplatz (Berlin),2,52,2.0,19.0,31.0,59.62,3.85,36.54,-144.71
3,Antonplatz (Berlin),3,75,1.0,20.0,54.0,72.00,1.33,26.67,-112.00
4,Antonplatz (Berlin),4,100,3.0,26.0,71.0,71.00,3.00,26.00,-75.00
5,Antonplatz (Berlin),5,123,4.0,38.0,81.0,65.85,3.25,30.89,-86.34
6,Antonplatz (Berlin),6,129,16.0,33.0,80.0,62.02,12.40,25.58,-72.56
7,Antonplatz (Berlin),7,149,27.0,36.0,86.0,57.72,18.12,24.16,-32.62
8,Antonplatz (Berlin),8,142,20.0,38.0,84.0,59.15,14.08,26.76,-35.92
9,Antonplatz (Berlin),9,148,10.0,35.0,103.0,69.59,6.76,23.65,-49.46


In [19]:
conn.execute("""
            WITH 
            combined_events AS (
            -- Arrivals
            SELECT
                'arrival' as event_type,
                trip_id,
                station_name,
                CAST(scheduled_time AS DATETIME) as scheduled_time,
                CAST(actual_time AS DATETIME) as actual_time,
                EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds
            FROM m13_arrivals
            WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
            
            UNION ALL
            
            -- Departures
            SELECT
                'departure' as event_type,
                trip_id,
                station_name,
                CAST(scheduled_time AS DATETIME) as scheduled_time,
                CAST(actual_time AS DATETIME) as actual_time,
                EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds
            FROM m13_departures
            WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
        ),
        
        punctuality_check AS (
            SELECT
                station_name,
                event_type,
                scheduled_time,
                actual_time,
                -- Extract the difference in seconds for analysis
                EXTRACT(EPOCH FROM actual_time - scheduled_time) as delay_seconds,
                CASE 
                    WHEN ABS(EXTRACT(EPOCH FROM actual_time - scheduled_time)) <= 60
                        THEN 'punctual'
                    WHEN EXTRACT(EPOCH FROM actual_time - scheduled_time) < -60 
                        THEN 'early'
                    ELSE 'late'
                END as punctuality_status,
                ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time 
                                    ORDER BY actual_time DESC) as row_num
            FROM combined_events
            )
            
        SELECT
            station_name,
            COUNT(*) as trips,
            SUM(CASE WHEN punctuality_status = 'late' THEN 1 ELSE 0 END) as late_trips,
            SUM(CASE WHEN punctuality_status = 'early' THEN 1 ELSE 0 END) as early_trips,
            SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END) as punctual_trips,
            (100 * SUM(CASE WHEN punctuality_status = 'punctual' THEN 1 ELSE 0 END)) / COUNT(*) as percent_punctual
        FROM punctuality_check
        WHERE row_num = 1
        GROUP BY station_name
        
        """).fetch_df()

,station_name,trips,late_trips,early_trips,punctual_trips,percent_punctual
0,Schönhauser Allee/Bornholmer Str. (Berlin),3187,521.0,1217.0,1449.0,45.465955
1,Antonplatz (Berlin),3099,287.0,869.0,1943.0,62.697644


## Preparing the end data set for a Data Analyst

In [21]:
final_df = conn.execute("""
                WITH combined_events AS (
                -- Arrivals
                SELECT
                    'arrival' as event_type,
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds
                FROM m13_arrivals
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
                
                UNION ALL
                
                -- Departures
                SELECT
                    'departure' as event_type,
                    trip_id,
                    station_name,
                    CAST(scheduled_time AS DATETIME) as scheduled_time,
                    CAST(actual_time AS DATETIME) as actual_time,
                    EXTRACT(EPOCH FROM (CAST(actual_time AS DATETIME) - CAST(scheduled_time AS DATETIME))) as delay_seconds
                FROM m13_departures
                WHERE CAST(scheduled_time AS DATE) NOT IN ('2025-01-23', '2025-01-27', '2025-02-08')
            ),
            
            analyzed_data AS (
                SELECT
                    trip_id,
                    station_name,
                    event_type,
                    scheduled_time,
                    actual_time,
                    delay_seconds,
                    -- Time components for easy analysis
                    EXTRACT(DOW FROM scheduled_time) as day_of_week,
                    EXTRACT(HOUR FROM scheduled_time) as hour_of_day,
                    CAST(scheduled_time AS DATE) as service_date,
                    -- Punctuality categorization
                    CASE 
                        WHEN ABS(delay_seconds) <= 60 THEN 'punctual'
                        WHEN delay_seconds < -60 THEN 'early'
                        ELSE 'late'
                    END as punctuality_status,
                    -- Delay buckets for more detailed analysis
                    CASE 
                        WHEN delay_seconds <= -300 THEN 'more_than_5min_early'
                        WHEN delay_seconds <= -60 THEN '1_to_5min_early'
                        WHEN delay_seconds <= 60 THEN 'on_time'
                        WHEN delay_seconds <= 300 THEN '1_to_5min_late'
                        ELSE 'more_than_5min_late'
                    END as delay_bucket,
                    -- Peak hours flag
                    CASE 
                        WHEN EXTRACT(HOUR FROM scheduled_time) BETWEEN 7 AND 9 THEN 'morning_peak'
                        WHEN EXTRACT(HOUR FROM scheduled_time) BETWEEN 16 AND 18 THEN 'evening_peak'
                        ELSE 'off_peak'
                    END as peak_status,
                    -- Weekend flag
                    CASE 
                        WHEN EXTRACT(DOW FROM scheduled_time) IN (0, 6) THEN 'weekend'
                        ELSE 'weekday'
                    END as day_type,
                    ROW_NUMBER() OVER(PARTITION BY trip_id, station_name, scheduled_time, event_type 
                                     ORDER BY actual_time DESC) as row_num
                FROM combined_events
            )
            
            SELECT 
                trip_id,
                station_name,
                event_type,
                scheduled_time,
                actual_time,
                delay_seconds,
                day_of_week,
                hour_of_day,
                service_date,
                punctuality_status,
                delay_bucket,
                peak_status,
                day_type
            FROM analyzed_data
            WHERE row_num = 1  -- Remove duplicates
            ORDER BY scheduled_time, station_name, event_type;
                """).df()

final_df.head(5)

,trip_id,station_name,event_type,scheduled_time,actual_time,delay_seconds,day_of_week,hour_of_day,service_date,punctuality_status,delay_bucket,peak_status,day_type
0,1|64498|2|86|23012025,Schönhauser Allee/Bornholmer Str. (Berlin),arrival,2025-01-23 23:08:00,2025-01-23 23:07:00,-60.0,4,23,2025-01-23,punctual,1_to_5min_early,off_peak,weekday
1,1|64498|2|86|23012025,Schönhauser Allee/Bornholmer Str. (Berlin),departure,2025-01-23 23:08:00,2025-01-23 23:07:00,-60.0,4,23,2025-01-23,punctual,1_to_5min_early,off_peak,weekday
2,1|64462|8|86|23012025,Schönhauser Allee/Bornholmer Str. (Berlin),arrival,2025-01-23 23:09:00,2025-01-23 23:07:00,-120.0,4,23,2025-01-23,early,1_to_5min_early,off_peak,weekday
3,1|64462|8|86|23012025,Schönhauser Allee/Bornholmer Str. (Berlin),departure,2025-01-23 23:09:00,2025-01-23 23:07:00,-120.0,4,23,2025-01-23,early,1_to_5min_early,off_peak,weekday
4,1|64498|3|86|23012025,Antonplatz (Berlin),arrival,2025-01-23 23:18:00,2025-01-23 23:19:00,60.0,4,23,2025-01-23,punctual,on_time,off_peak,weekday


In [22]:
final_df.to_csv('BVG_m13_data.csv')